<a href="https://colab.research.google.com/github/gupta24789/sentiment-analysis/blob/main/sentiment_rnn_lighting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !pip install -q lightning
# !pip install -q neattext

In [3]:
import pandas as pd
import numpy as np
import neattext as nt
import itertools
import warnings

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.nn.utils.rnn import  pad_sequence
from torch.utils.data import Dataset, DataLoader, TensorDataset

from torchmetrics import Accuracy
import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint

warnings.filterwarnings('ignore')

In [4]:
train_df = pd.read_csv("https://raw.githubusercontent.com/gupta24789/sentiment-analysis/main/data/train.csv")
val_df = pd.read_csv("https://raw.githubusercontent.com/gupta24789/sentiment-analysis/main/data/val.csv")

train_df = train_df[['raw_tweet', 'label']].dropna().reset_index(drop = True)
val_df = val_df[['raw_tweet', 'label']].dropna().reset_index(drop = True)